In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

In [2]:
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist
import pandas as pd

In [3]:
physical_devices =  tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [5]:
# HYPERPARAMETERS
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001

In [16]:
train_df = pd.read_csv("Multi_Digit_Mnist/train.csv")
test_df = pd.read_csv("Multi_Digit_Mnist/test.csv")
train_images = os.getcwd() + "/Multi_Digit_Mnist/train_images/" + train_df.iloc[:, 0].values
test_images = os.getcwd() + "/Multi_Digit_Mnist/test_images/" + test_df.iloc[:, 0].values

train_labels = train_df.iloc[:, 1:].values
test_labels = test_df.iloc[:, 1:].values


def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    # In older versions you need to set shape in order to avoid error
    # on newer (2.3.0+) the following 3 lines can safely be removed
    image.set_shape((64, 64, 1))
    label[0].set_shape([])
    label[1].set_shape([])

    labels = {"first_num": label[0], "second_num": label[1]}
    return image, labels


AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = (
    test_dataset.map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

In [17]:
#Functional API
inputs = keras.Input(shape=(64,64,1))
x = layers.Conv2D(
    filters=32,
    kernel_size=3,
    padding="same",
    kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
)(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.Conv2D(64, 3, kernel_regularizer=regularizers.l2(WEIGHT_DECAY),)(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(
    64, 3, activation="relu", kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
)(x)
x = layers.Conv2D(128, 3, activation = 'relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation = 'relu')(x)
output1 = layers.Dense(10, activation = 'softmax', name='first_num')(x)
output2 = layers.Dense(10, activation = 'softmax', name='second_num')(x)

model = keras.Model(inputs=inputs, outputs = [output1, output2])

model.compile(
    optimizer=keras.optimizers.Adam(LEARNING_RATE),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

model.fit(train_dataset, epochs=5, verbose=2)
model.evaluate(test_dataset, verbose=2)

Epoch 1/5
1000/1000 - 59s - loss: 1.7979 - first_num_loss: 0.8611 - second_num_loss: 0.8554 - first_num_accuracy: 0.6983 - second_num_accuracy: 0.6987 - 59s/epoch - 59ms/step
Epoch 2/5
1000/1000 - 68s - loss: 0.5133 - first_num_loss: 0.2254 - second_num_loss: 0.2233 - first_num_accuracy: 0.9263 - second_num_accuracy: 0.9274 - 68s/epoch - 68ms/step
Epoch 3/5
1000/1000 - 56s - loss: 0.3441 - first_num_loss: 0.1466 - second_num_loss: 0.1405 - first_num_accuracy: 0.9533 - second_num_accuracy: 0.9551 - 56s/epoch - 56ms/step
Epoch 4/5
1000/1000 - 50s - loss: 0.2653 - first_num_loss: 0.1069 - second_num_loss: 0.1054 - first_num_accuracy: 0.9660 - second_num_accuracy: 0.9659 - 50s/epoch - 50ms/step
Epoch 5/5
1000/1000 - 48s - loss: 0.2230 - first_num_loss: 0.0905 - second_num_loss: 0.0837 - first_num_accuracy: 0.9721 - second_num_accuracy: 0.9731 - 48s/epoch - 48ms/step
313/313 - 7s - loss: 0.9414 - first_num_loss: 0.3218 - second_num_loss: 0.5720 - first_num_accuracy: 0.9097 - second_num_accu

[0.9414374828338623,
 0.32180291414260864,
 0.5720217227935791,
 0.9096500277519226,
 0.8511499762535095]